In [1]:
import xarray as xr
from pathlib import Path
from typing import List, Dict, Tuple, NamedTuple, Iterable, Hashable
from datetime import date, timedelta
import matplotlib.pyplot as plt
import numpy.typing as npt
import numpy as np
import cartopy.crs as ccrs
from tqdm import tqdm
from plotting import date_to_processed, processed_to_date, make_png

In [2]:
folders = [
    Path(r"C:\Users\Casper\OneDrive - Danmarks Tekniske Universitet\SKOLE\Kandidat\Syntese\ProcessedGrids\Processed_v4_v10"),
    Path(r"C:\Users\Casper\OneDrive - Danmarks Tekniske Universitet\SKOLE\Kandidat\Syntese\ProcessedGrids\Processed_v5_v01"),
    Path(r"C:\Users\Casper\OneDrive - Danmarks Tekniske Universitet\SKOLE\Kandidat\Syntese\ProcessedGrids\v1"),
    Path(r"C:\Users\Casper\OneDrive - Danmarks Tekniske Universitet\SKOLE\Kandidat\Syntese\ProcessedGrids\v2"),
    Path(r"C:\Users\Casper\OneDrive - Danmarks Tekniske Universitet\SKOLE\Kandidat\Syntese\ProcessedGrids\v3"),
    Path(r"C:\Users\Casper\OneDrive - Danmarks Tekniske Universitet\SKOLE\Kandidat\Syntese\ProcessedGrids\v5"),
    Path(r"C:\Users\Casper\OneDrive - Danmarks Tekniske Universitet\SKOLE\Kandidat\Syntese\ProcessedGrids\v6"),
    Path(r"C:\Users\Casper\OneDrive - Danmarks Tekniske Universitet\SKOLE\Kandidat\Syntese\ProcessedGrids\v7"),
    Path(r"C:\Users\Casper\OneDrive - Danmarks Tekniske Universitet\SKOLE\Kandidat\Syntese\ProcessedGrids\v8"),
    Path(r"C:\Users\Casper\OneDrive - Danmarks Tekniske Universitet\SKOLE\Kandidat\Syntese\ProcessedGrids\v9"),
]

choosen_date = date(2019, 1, 2)
tds = [0.1, 1, 0.05, 0.005, 0.0005, 0.00005, 5, 0.5, 2, 20]

In [3]:
base = Path(r"C:\Users\Casper\OneDrive - Danmarks Tekniske Universitet\SKOLE\Kandidat\Syntese\ProcessedGrids\Processed_v4_v10")
dates = [base / date_to_processed(date(2019,1,2) - timedelta(i)) for i in range(-5,6)]

In [6]:
features = ['sla', 'wind_speed', 'sst', 'swh']
mults = [100, 1, 1, 1]
units = ["cm", "m/s", f"\N{DEGREE SIGN}C", "m"]
cbars = [(-75, 75), (-1, 15), (-1, 30), (0, 6)]
output_folder = Path("Grids")

for feature, unit, cbar, mult in zip(features, units, cbars, mults):
    (output_folder / feature).mkdir(parents=True, exist_ok=True)
    for td, folder in zip(tds, folders):
        file = folder / date_to_processed(choosen_date)
        image = xr.open_dataset(file, engine='netcdf4')
        if feature not in list(image.data_vars):
            continue
        make_png(
            image = image[feature] * mult,
            figsize = (13, 6),
            output_path = output_folder / feature / f"{feature}_{file.name.replace('.nc', '')}_{td*100000:.0f}.png",
            extent = [-180, 180, -90, 90],
            vmin = cbar[0],
            vmax = cbar[1],
            title = f"{processed_to_date(file)} - Time weighting = {td}",
            cbar_label = f"{feature.upper()} [{unit}]",
            fontsize = 17,
            ticksize = 15,
            titlesize = 20
        )